# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data importing
Helper function that will help transforming the string that represents a list in the Excel file, to an actual Python list

In [2]:
def string_to_list(column):
    return [element.strip(" ") for element in column.strip("[]").replace("'", "").replace('"', "").split(", ")]

def single_list_string_to_string(column):
    return column.replace("'", "").replace('"', "").replace("[", "").replace("]", "")

In [3]:
PATH = 'data/'

recensement = pd.read_excel(PATH + '1832_cleaned_data_recensement.xlsx')
recensement["PROPRIÉTAIRES DES MAISONS."] = recensement["PROPRIÉTAIRES DES MAISONS."].apply(string_to_list)
recensement["NOMS DES RUES."] = recensement["NOMS DES RUES."].apply(single_list_string_to_string)
recensement["VOCATION."] = recensement["VOCATION."].apply(single_list_string_to_string)
recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME."] = recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME."].apply(single_list_string_to_string)
recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE."] = recensement["NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE."].apply(single_list_string_to_string)

cadastre = pd.read_csv(PATH + 'parcelInfos.csv').query("folio <= 11")
cadastre["nr"] = cadastre["nr"].astype("int32")
cadastre["use"] = cadastre["use"].apply(single_list_string_to_string)

folio_links = pd.read_excel(PATH + "Folio_link.xlsx")

In [4]:
recensement.head(2)

,Unnamed: 0,NOMS DES RUES.,N° des MAISONS.,PROPRIÉTAIRES DES MAISONS.,NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME.,NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE.,Année de la naissance. [h],NOMS DE LEURS ÉPOUSES.,Année de la naissance. [f],Noms de leurs enfans actuellement dans la Commune.,...,Date de leur demeure dans la Commune.,VOCATION.,N° de leur Récépissé. [famille],"NOMS DES PENSIONNAIRES, COMMIS, APPRENTIS, OUVRIERS ET DOMESTIQUES. NOMS DE BAPTÊME.","NOMS DES PENSIONNAIRES, COMMIS, APPRENTIS, OUVRIERS ET DOMESTIQUES. NOMS DE FAMILLE.","D'où ils sont bourgeois, ou à ce défaut d'où ils sont originaires. [dom/pens]","Indiquer ici s'ils sont Pensionnaires, Commis, Apprentis, Ouvriers ou Domestiques.",N° de leur Récépissé. [dom/pens],OBSERVATIONS.,Page
0,0,Martherey,['7'],[Gilliéron],Jean Louis,"Deprez, Devez",['1768'],"[['Echs', 'Ochs']]",['1793'],"[['Laura', 'Lucia', 'Lucy']]",...,['1831'],journalier,[],[],[],[],[],[],[],3
1,1,Martherey,['7'],[],"Jean Andre, Jean Marc, Jean Marie, Jeanine, Je...",Fontannaz,['1778'],"['Sophprant', 'Loretti']",['1778'],"['Jeanne', 'Jeanne', 'Abram', 'Jeannette', ['D...",...,['1812'],journalier,['1525'],[],[],[],[],[],[],3


In [50]:
cadastre

,Unnamed: 0,Unnamed: 0.1,folio,nr,articles,use,owner_surname,Unnamed: 6,pos,street,owner_names
0,0,0,1,1,NaN,"place, passage",Commune de Lausanne,NaN,"[46.5194648115546, 6.6302086723144145]",[Grand Chêne],"[Commune, de, Lausanne]"
1,1,1,1,2,NaN,terrasse,Commune de Lausanne,NaN,"[46.5194836242143, 6.63041722290751]",[Grand Chêne],"[Commune, de, Lausanne]"
2,2,2,1,3,NaN,place,Marie-Antoine De la Forêt,NaN,"[46.51964083504432, 6.630692981092635]",[Grand Chêne],"[Marie-Antoine, De, la, Forêt]"
3,3,3,1,4,NaN,maison,Marie-Antoine De la Forêt,NaN,"[46.519611915812376, 6.630816905582044]",[Grand Chêne],"[Marie-Antoine, De, la, Forêt]"
4,4,4,1,5,NaN,jardin,Marie-Antoine De la Forêt,NaN,"[46.519533178353655, 6.630660454149068]",[Grand Chêne],"[Marie-Antoine, De, la, Forêt]"
...,...,...,...,...,...,...,...,...,...,...,...
2384,2384,2264,11,151,NaN,remise,Jean-Louis Hugonet,NaN,"[46.523066826644694, 6.628385347669797]",[],"[Jean-Louis, Hugonet]"
2385,2385,2265,11,152,NaN,bucher,Jean-Louis Hugonet,NaN,"[46.522951838571174, 6.628176643863636]",[],"[Jean-Louis, Hugonet]"
2386,2386,2266,11,153,NaN,maison,Jean-Louis Hugonet,NaN,"[46.52282275384253, 6.628169770760005]",[],"[Jean-Louis, Hugonet]"
2387,2387,2267,11,154,NaN,jardin,Jean-Louis Hugonet,NaN,"[46.523062484487525, 6.62825034740809]",[],"[Jean-Louis, Hugonet]"


In [6]:
folio_links.head(2)

,folio,rue,num_start,num_stop
0,1,Grand Chêne,1,83
1,1,Petit Chêne,28,46


In [19]:
data = []
cadastre["street"] = [[] for _ in range(len(cadastre))]
for i, row in folio_links.iterrows():
    folio = row["folio"]
    for num in range(row["num_start"], row["num_stop"]):
        try:
            for i in range(len(cadastre.loc[(cadastre["nr"] == num) & (cadastre["folio"] == folio), "street"].values)):
                cadastre.loc[(cadastre["nr"] == num) & (cadastre["folio"] == folio), "street"].values[i].append(row["rue"])
        except:
            print(cadastre[(cadastre["nr"] == num) & (cadastre["folio"] == folio)]["street"], folio, num)
cadastre["street"] = cadastre["street"].apply(lambda x : list(set(x)))

In [8]:
cadastre.loc[20:30]

,Unnamed: 0,Unnamed: 0.1,folio,nr,articles,use,owner_surname,Unnamed: 6,pos,street
20,20,20,1,21,NaN,hangar,Marie-Susanne Dapples née Gleyre,NaN,"[46.51941682201014, 6.6315477198889266]",[Grand Chêne]
21,21,21,1,22,NaN,place,Marie-Susanne Dapples née Gleyre,NaN,"[46.51945439351096, 6.631568242904952]",[Grand Chêne]
22,22,22,1,23,NaN,"bucher, serre",Marie-Susanne Dapples née Gleyre,NaN,NaN,[Grand Chêne]
23,23,23,1,24,NaN,maison,Marie-Susanne Dapples née Gleyre,NaN,"[46.519552248080515, 6.631586861933149]",[Grand Chêne]
24,24,24,1,23,NaN,cabinet,Louis-Benjamin Rouge,NaN,NaN,[Grand Chêne]
25,25,25,1,25,NaN,maison,Louis-Benjamin Rouge,NaN,"[46.51956142095431, 6.631707205101971]",[Grand Chêne]
26,26,26,1,26,NaN,jardin,Louis-Benjamin Rouge,NaN,"[46.51943103153396, 6.631694091524555]",[Grand Chêne]
27,27,27,1,27,NaN,bucher,Jean-Louis Mellet,NaN,"[46.519059860415204, 6.631348235175288]",[Grand Chêne]
28,28,28,1,28,NaN,jardin,Jean-Louis Mellet,NaN,"[46.51903708438157, 6.631556401511808]","[Grand Chêne, Petit Chêne]"
29,29,29,1,29,NaN,maison,Jean-Louis Mellet,NaN,"[46.519110819709354, 6.631691843619361]","[Grand Chêne, Petit Chêne]"


# Merging the two datasets
Before trying to merge, we must "explode" the recensement dataset such that we get one entry for each owner. Meaning that if an entry has owner [A, B], it should create two entries, the first one belonging to A only and the second one to B only. We then only keep an entry if it has an owner

In [20]:
recensement_exploded = recensement.explode("PROPRIÉTAIRES DES MAISONS.")
recensement_exploded = recensement_exploded[recensement_exploded["PROPRIÉTAIRES DES MAISONS."].str.len() > 0]

Because of this flaw, let's try to split the owner names on blank spaces in the cadastre dataset and explode it as well

In [21]:
cadastre["owner_names"] = cadastre["owner_surname"].apply(lambda name : name.split(" "))
cadastre_exploded = cadastre.explode("owner_names")
cadastre_exploded = cadastre_exploded.explode("street")
cadastre_exploded.loc[30:50]

,Unnamed: 0,Unnamed: 0.1,folio,nr,articles,use,owner_surname,Unnamed: 6,pos,street,owner_names
30,30,30,1,30,NaN,maison,Jaques-Louis Berger,NaN,"[46.51917700375838, 6.631740051334762]",Grand Chêne,Jaques-Louis
30,30,30,1,30,NaN,maison,Jaques-Louis Berger,NaN,"[46.51917700375838, 6.631740051334762]",Petit Chêne,Jaques-Louis
30,30,30,1,30,NaN,maison,Jaques-Louis Berger,NaN,"[46.51917700375838, 6.631740051334762]",Grand Chêne,Berger
30,30,30,1,30,NaN,maison,Jaques-Louis Berger,NaN,"[46.51917700375838, 6.631740051334762]",Petit Chêne,Berger
31,31,31,1,31,NaN,maison,Jaques-Louis Berger,NaN,"[46.519226548862484, 6.631787503526648]",Grand Chêne,Jaques-Louis
...,...,...,...,...,...,...,...,...,...,...,...
49,49,48,1,48,NaN,buanderie,Marie-Antoine De la Forêt,NaN,"[46.51981675515511, 6.63077368419727]",Grand Chêne,Forêt
50,50,49,1,49,NaN,bois,Marie-Antoine De la Forêt,NaN,"[46.52000311197292, 6.63066547811636]",Grand Chêne,Marie-Antoine
50,50,49,1,49,NaN,bois,Marie-Antoine De la Forêt,NaN,"[46.52000311197292, 6.63066547811636]",Grand Chêne,De
50,50,49,1,49,NaN,bois,Marie-Antoine De la Forêt,NaN,"[46.52000311197292, 6.63066547811636]",Grand Chêne,la


In [31]:
merged = pd.merge(cadastre_exploded, recensement_exploded, left_on = ["owner_names", "street"], right_on = ["PROPRIÉTAIRES DES MAISONS.", "NOMS DES RUES."], how="inner")
print(len(merged))
print(len(merged.drop_duplicates(["nr", "folio"])))

1402
718


In [49]:
dataset = merged[["nr", "folio", "owner_surname", "pos", "street", "NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE BAPTÊME.", "NOMS DES CHEFS DE FAMILLE ET AUTRES LOCATAIRES. NOMS DE FAMILLE.", "VOCATION.", "use"]].dropna()
dataset.columns = ["nr", "folio","owner", "position", "street", "renter_forname", "renter_lastname", "job", "use"]
dataset.position = dataset.position.apply(lambda pos : [float(p) for p in string_to_list(pos)])
print(len(dataset.loc[dataset.query("use == 'maison'").drop_duplicates(["folio", "nr"]).index]) / len(dataset.query("use == 'maison'")))
len(dataset)

0.48091603053435117


1359

,nr,folio,owner,position,street,renter_forname,renter_lastname,job,use
0,12,1,Auguste-César Senarclens,"[46.51960267206674, 6.631076076197757]",Grand Chêne,Monsieur le Chevalier,dttotcher,rentier,maison
1,13,1,Auguste-César Senarclens,"[46.51951074701437, 6.63101419748413]",Grand Chêne,Monsieur le Chevalier,dttotcher,rentier,"aisances, place"
2,14,1,Auguste-César Senarclens,"[46.51942590220815, 6.631070685020248]",Grand Chêne,Monsieur le Chevalier,dttotcher,rentier,jardin
3,17,1,Jean-Henri Favez,"[46.5195598497228, 6.6314863483443816]",Grand Chêne,Henry,877704,,maison
4,18,1,Jean-Henri Favez,"[46.51942050853854, 6.631481400953227]",Grand Chêne,Henry,877704,,place
...,...,...,...,...,...,...,...,...,...
1397,300,8,"enfans Samuel-Jaques Hollard (Alexandre-Louis,...","[46.52161986127709, 6.633653884670923]",Palud,Sigismond,Curtat,rentier,cour
1398,301,8,"enfans Samuel-Jaques Hollard (Alexandre-Louis,...","[46.52160348694391, 6.633706535807181]",Palud,Sigismond,Curtat,rentier,buanderie
1399,302,8,Paul-Pierre Gély,"[46.52164814630788, 6.633674814577782]",Palud,"Benoit, Bonnet, Honorat, Honoré","Bally, Delay, Delley, Dülly, Fell, Felli, Gall...",,buanderie
1400,303,8,Paul-Pierre Gély,"[46.52167352472374, 6.633663761073975]",Palud,"Benoit, Bonnet, Honorat, Honoré","Bally, Delay, Delley, Dülly, Fell, Felli, Gall...",,cour


In [44]:
dataset.to_json("website/data.json", orient="records")

In [36]:
cadastre.query("owner_surname == 'Jeanne-Françoise Guisan née Dapples'")

,Unnamed: 0,Unnamed: 0.1,folio,nr,articles,use,owner_surname,Unnamed: 6,pos,street,owner_names
1153,1153,1111,6,151,NaN,maison,Jeanne-Françoise Guisan née Dapples,NaN,"[46.52337610743361, 6.635635174087884]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1154,1154,1112,6,152,NaN,cour,Jeanne-Françoise Guisan née Dapples,NaN,"[46.52331763087088, 6.6356203098065585]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1155,1155,1112,6,152,NaN,cour,Jeanne-Françoise Guisan née Dapples,NaN,"[46.523923087883446, 6.634665356724987]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1156,1156,1113,6,153,NaN,maison,Jeanne-Françoise Guisan née Dapples,NaN,"[46.523267938733994, 6.635504033866326]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1157,1157,1114,6,154,NaN,buanderie,Jeanne-Françoise Guisan née Dapples,NaN,"[46.523191099868434, 6.6353881615549755]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
1158,1158,1115,6,155,NaN,cour,Jeanne-Françoise Guisan née Dapples,NaN,"[46.523176124028616, 6.635477865389208]","[Cité Derrière, Cité Devant]","[Jeanne-Françoise, Guisan, née, Dapples]"
